# 1.LIBRARIES ------------------------------------------------

In [ ]:
!pip install sentencepiece==0.1.99 transformers==4.33.2 datasets sacremoses  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 39.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00


In [ ]:
import torch
import random
import re
import pandas as pd
import numpy as np
import sentencepiece.sentencepiece_model_pb2 as sp_pb2_model
from collections import Counter, defaultdict
from tqdm.auto import tqdm, trange
from transformers import AutoModelForSeq2SeqLM
from transformers import NllbTokenizer
import sys
import unicodedata
from sacremoses import MosesPunctNormalizer
import sentencepiece as spm

# 2.CONSTANTS ------------------------------------------------

In [ ]:
from google.colab import drive
drive.mount('/Til')

Mounted at /Til


In [ ]:
PATH = '/Til/MyDrive/Colab Notebooks/Til/'
DATA_PATH = "".join([PATH, 'Data/'])
MODEL_NAME = "facebook/nllb-200-distilled-600M"
ALL_SENTENCES_PATH = "".join([DATA_PATH, 'All_model.csv'])
MODEL_PATH = "".join([PATH, 'models/NLLB_v1/'])
# OLD_TOKENIZER_PATH = "".join([MODEL_PATH, 'old_tokenizer'])
# OLD_TOKENIZER_BPE_PATH = "".join([OLD_TOKENIZER_PATH, "/sentencepiece.bpe.model"])
LANG_UNICODE = 'krc_Cyrl'
MODEL_PATH_RAW = "".join([MODEL_PATH, 'nllb_krc_raw'])

ALL_TEXT_PLAIN = "".join([DATA_PATH, 'my_texts_plain.txt'])
SPM_PREFIX = "".join([MODEL_PATH, 'SPM_QM/SPM_QM'])
NEW_SPM_NAME = "".join([MODEL_PATH, 'SPM_QM/spm_nllb_qm.model'])

PUNCT = '.,-—:)(»«!?–/;„"“…*́№Ёҥ[]”^%+І=і•_􏰀²|}{#‘■>⁠’á<°\§\''

# 3.DOWNLOADING ------------------------------------------------

In [ ]:
all_sentences = pd.read_csv(ALL_SENTENCES_PATH, sep = ';')
all_sentences_krc = np.array(all_sentences.krc.replace(PUNCT, ""))
print(all_sentences_krc[:10])

['Теnиз тузлу некди?'
 'Бир jашчыq анасы бла тиnизге jуwунурgа барgанды. Jашчыq qуwаныb, суwgа кёмюлсе, аwузуна суw киргенди, кеси wa тузлу суw. Jунчуgан да этиb, бир уртлам суwну да jутуb, ол jаgаgа чыqgанды. Суwну алай тузлу нек болgанын аnыламай:'
 '- Некди теnиз былай тузлу? – деb сейирсинnенди.'
 'Анасы jашына теnизни хапарын айтханды:'
 '- Эртде-эртде бир бай qарнаш бла бир jарлы qарнаш jашаgандыла. Jарлы qарнаш байgа «менnе муну бла бир тюйчюк бер» деb, табаqны алыb, тилей барgанды. Бай qарнаш бермегенди.'
 'Jарлы, аны юйюнден чыgыb, мыдах болуb, келе турgанлай, qолунда гырjынчыgы бла бир киши чыqgанды аллына. '
 '- Не айланаса? – деb сорgан кишичик.'
 'Jаш аnа, былай да былай деb, jарлылыq хапарын айтханды.'
 '- Ма, бу гырjынчыqны ал да, таw артына бар. Анда гитче адамчыqла jашайдыла. Башха затха унамай, гырjынчыgыnы тирменчикге аwушдур. Андан арысын кёре барырса, - деb кишичик гырjынчыgын да jашха тутдуруb, jолуна кетеди.'
 'Jарлы qарнаш таw артына аwgанды, ол адамчыqлаgа тюлб

In [ ]:
tokenizer = NllbTokenizer.from_pretrained(MODEL_NAME)

# 4.LOOKING AT THE DATA ------------------------------------------------

### How well does the data fit into a NLLB tokenizer?

In [ ]:
def wordTokenize(text):
    # a very naive word tokenizer for languages with English-like orthography
    return re.findall('(\\w+|[^\\w\\s])', text)

# smpl = all_sentences.sample(10000, random_state=1)
smpl = all_sentences
smpl['rus_toks'] = smpl.rus.apply(tokenizer.tokenize)
smpl['krc_toks'] = smpl.krc.apply(tokenizer.tokenize)
smpl['rus_words'] = smpl.rus.apply(wordTokenize)
smpl['krc_words'] = smpl.krc.apply(wordTokenize)

print(smpl)

                                                      krc  \
0                                      Теnиз тузлу некди?   
1       Бир jашчыq анасы бла тиnизге jуwунурgа барgанд...   
2         - Некди теnиз былай тузлу? – деb сейирсинnенди.   
3                  Анасы jашына теnизни хапарын айтханды:   
4       - Эртде-эртде бир бай qарнаш бла бир jарлы qар...   
...                                                   ...   
260597               Оgай, алай тюлдю, сиз артда билирсиз   
260598  Мен заман бла ант этеме, Толу ийнаныw, 2. Ашхы...   
260599      Сизни динигиз сизге, мени диним менnе», - де.   
260600  Осебебден махтаwла бла Раббиnи тазала, кечмекл...   
260601     Абу-Лахабны эки qолу qурусун, кеси да qурусун!   

                                                      rus  \
0                                    Почему море солёное?   
1       Один мальчик со своей мамой на море купаться п...   
2       - Почему вода такая солёная? – он поинтересова...   
3                   Мам

Actually, we can compute precise statistics of this:

In [ ]:
stats = smpl[
    ['rus_toks', 'krc_toks', 'rus_words', 'krc_words']
].applymap(len).describe()
print(stats.rus_toks['mean'] / stats.rus_words['mean'])  # 1.65
print(stats.krc_toks['mean'] / stats.krc_words['mean'])  # 2.38

1.6593475518751215
2.3755470642360565


In [ ]:
print(stats)

            rus_toks       krc_toks      rus_words      krc_words
count  260602.000000  260602.000000  260602.000000  260602.000000
mean       13.775873      18.545679       8.301982       7.806909
std        29.168188      38.949462      19.266522      17.671651
min         1.000000       1.000000       1.000000       1.000000
25%         3.000000       4.000000       1.000000       1.000000
50%         5.000000       7.000000       2.000000       2.000000
75%        11.000000      14.000000       7.000000       6.000000
max      2019.000000    2506.000000    1277.000000    1072.000000


There is enough big range between a number of tokens for russian and qarachay-malqar languages. So we need to add some tokens to nllb vocabulary

### CHECK UNK NUMBER

In [ ]:
texts_with_unk = [
    text for text in tqdm(all_sentences.krc)
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]
print(len(texts_with_unk))
s = random.sample(texts_with_unk, 5)

  0%|          | 0/260602 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1432 > 1024). Running this sequence through the model will result in indexing errors


21642


We can see that out of 260 602 texts, 21642 contain an “unknown symbol” after tokenization.  
Most of these cases seem to be associated with non-standard punctuation marks, and there is a reason for that:   
* the NLLB team preprocessed their texts before training the tokenizer and the model.
* The code for preprocessing (adapted from the Stopes repo) looks like this:

In [ ]:
mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]

def get_non_printing_char_replacer(replace_by: str = " "):
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        # same as \p{C} in perl
        # see https://www.unicode.org/reports/tr44/#General_Category_Values
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    # replace 𝓕𝔯𝔞𝔫𝔠𝔢𝔰𝔠𝔞 by Francesca
    clean = unicodedata.normalize("NFKC", clean)
    return clean

In [ ]:
texts_with_unk_normed = [
    text for text in tqdm(texts_with_unk)
    if tokenizer.unk_token_id in tokenizer(preproc(text)).input_ids
]
print(len(texts_with_unk_normed))  # 0

  0%|          | 0/21642 [00:00<?, ?it/s]

0


In this case we can say that unk tokens it is just a punctuation.

# 5.EXPANDING VOCABULARY ------------------------------------------------

In [ ]:
chars_cnt = Counter(c for t in all_sentences_krc for c in t)

with open(ALL_TEXT_PLAIN, 'w') as f:
    for i, text in enumerate(all_sentences_krc):
        print(text, file=f)

I chose the vocabulary size to be 8K intuitively, because such a number of tokens can potentially cover the most important roots and suffixes in the language (to compare: NLLB vocabulary for 200 languages has 256000 tokens, but many of them are used by a lot of different languages).  
All the other parameters are not very important.

In [ ]:
spm.SentencePieceTrainer.train(
    input=ALL_TEXT_PLAIN,
    model_prefix=SPM_PREFIX,
    vocab_size=2**13,  # 8K
    character_coverage = 1,
    # num_threads=16,
    train_extremely_large_corpus=False,
    add_dummy_prefix=False,
    max_sentencepiece_length=512,
    max_sentence_length=4192*4,
    pad_id=0,
    eos_id=1,
    unk_id=2,
    bos_id=-1,
    required_chars=''.join([k for k, v in chars_cnt.most_common() if v >= 3 and k not in ' ']),
)

After training a QM tokenizer, I perform a “surgical operation” with it: extracting the sentencepiece model from the standard NLLB tokenizer and enriching it from all tokens from the QM tokenizer that have been missing from the NLLB tokenizer (based on the example from the sentencepiece repo).

reading the NLLB and the QM sentencepiece models into a native format

In [ ]:
sp_trained = spm.SentencePieceProcessor(model_file=f'{SPM_PREFIX}.model')
added_spm = sp_pb2_model.ModelProto()
added_spm.ParseFromString(sp_trained.serialized_model_proto())
old_spm = sp_pb2_model.ModelProto()
old_spm.ParseFromString(tokenizer.sp_model.serialized_model_proto())

# adding the missing tokens to the NLLB sentencepiece model
nllb_tokens_set = {p.piece for p in old_spm.pieces}
print(len(nllb_tokens_set))

256000


In [ ]:
prev_min_score = old_spm.pieces[-1].score
for p in added_spm.pieces:
    piece = p.piece
    if piece not in nllb_tokens_set:
        new_p = sp_pb2_model.ModelProto().SentencePiece()
        new_p.piece = piece
        # for all new tokens, I'll set a lower score (priority)
        new_p.score = p.score + prev_min_score
        old_spm.pieces.append(new_p)

print(len(old_spm.pieces))

262721


In [ ]:
# saving the result to disk
with open(NEW_SPM_NAME, 'wb') as f:
    f.write(old_spm.SerializeToString())

Finally, I need to update the neural network weights:
* add new embeddings for the freshly added tokens.
* in NLLB, the token embeddings reside in the parameter called shared.
It is used both in the encoder and decoder input embeddings and in the last decoder layer that predicts the distribution of the next token.  
  
By default, the embeddings for the new tokens are initialized randomly.   
Instead, I re-initialize each one with the average of the embeddings of the old tokens that corresponded to the new token (or if there are none, with the embedding of the <unk> token). This slightly improves the training speed, because the newly created tokken embeddings are already informative.

# 6.UPDATING TOKENIZER VOCABLUARY ------------------------------------------------

In [ ]:
tokenizer_old = NllbTokenizer.from_pretrained(MODEL_NAME)
print(len(tokenizer_old))
print(tokenizer_old.convert_ids_to_tokens([256202, 256203]))

256204
['zul_Latn', '<mask>']


In [ ]:
tokenizer = NllbTokenizer.from_pretrained(MODEL_NAME, vocab_file=NEW_SPM_NAME)

In [ ]:
print(len(tokenizer))
print(tokenizer.convert_ids_to_tokens([262923, 262924]))

262925
['zul_Latn', '<mask>']


In [ ]:
print(tokenizer.vocab_size)

262925


In [ ]:
n_new_tokens = len(tokenizer) - len(tokenizer_old)
print(n_new_tokens)

6721


In [ ]:
def fix_tokenizer(tokenizer, new_lang=LANG_UNICODE):
    """ Add a new language token to the tokenizer vocabulary (this should be done each time after its initialization) """
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [ ]:
fix_tokenizer(tokenizer)

### Check

In [ ]:
print(tokenizer.convert_ids_to_tokens([262923, 262924, 262925])) # ['zul_Latn', 'krc_Cyrl', '<mask>']
print(tokenizer.convert_tokens_to_ids(['zul_Latn', 'krc_Cyrl', '<mask>'])) # [262923, 262924, 262925]

['zul_Latn', 'krc_Cyrl', '<mask>']
[262923, 262924, 262925]


In [ ]:
print(len(tokenizer_old), len(tokenizer)) # 256204, 268559
added_vocab = set(tokenizer.get_vocab()).difference(set(tokenizer_old.get_vocab()))
print(len(added_vocab))  # 6721

256204 262926
6721


In [ ]:
text = random.choice(all_sentences_krc)

print(tokenizer_old.tokenize(text))
print(tokenizer.tokenize(text))

['▁Ч', 'ома', 'рт', '▁q', 'ой', 'чу', '▁деген', '▁j', 'а', 'ша', 'w', '-', 'тур', 'му', 'ш', '▁j', 'ома', 'q', 'да', '▁бир', '▁ё', 'кс', 'юз', '▁q', 'ыз', 'ны', '▁q', 'а', 'дары', 'ны', '▁ю', 'с', 'юн', 'ден', '▁айты', 'лады', ',', '▁А', 'n', 'а', '▁бир', '▁та', 'лай', '▁кере', '▁аман', 'лы', 'q', 'чы', 'ланы', '▁хыя', 'нат', 'лары', '▁j', 'ете', 'ди', ',', '▁А', 'лай', '▁ол', '▁хар', '▁заманда', '▁q', 'ы', 'йын', '▁болу', 'м', 'ла', 'дан', ',', '▁бе', 'тин', '▁тас', '▁эт', 'мей', ',', '▁чы', 'g', 'ы', 'b', '▁тура', 'ды', '.']
['▁Ч', 'ома', 'рт', '▁q', 'ойчу', '▁деген', '▁j', 'аша', 'w', '-', 'турмуш', '▁j', 'ома', 'q', 'да', '▁бир', '▁ё', 'кс', 'юз', '▁q', 'ызны', '▁q', 'а', 'дары', 'ны', '▁юсюнден', '▁айтылады', ',', '▁А', 'n', 'а', '▁бир', '▁талай', '▁кере', '▁аманлы', 'q', 'чыланы', '▁хыянат', 'лары', '▁j', 'етеди', ',', '▁Алай', '▁ол', '▁хар', '▁заманда', '▁q', 'ыйын', '▁болум', 'ладан', ',', '▁бетин', '▁тас', '▁этмей', ',', '▁чы', 'g', 'ы', 'b', '▁турады', '.']


In [ ]:
random.seed(1)
sample = random.sample(list(all_sentences_krc), 10000)
sample[:10]

['Алай болgанлыqgа тиширыw оюмлулуq бла ийманда, сюймекликде эмда сыйлылыqда турса, Сабийниa табыwуну юсю бла qутхарыллыqды. Jамаgатны тамадалары.',
 'см протекать',
 'Унипикачияларgа',
 'Балтаны сабларgа',
 'Мени qартлыqыма qарамаgыз',
 'Ол qалайgа барса да, qурgаq отун этиb кегеди.',
 'бёлюнмей',
 'Кюндюз да, сен ары-бери jюзюw кёбдюле.',
 'сов кого-что qучаqларgа',
 'Qанат\xadлычы']

In [ ]:
pd.DataFrame({
    'old': [len(tokenizer_old.tokenize(text)) for text in sample],
    'new': [len(tokenizer.tokenize(text)) for text in sample]
}).describe()

,old,new
count,10000.000000,10000.000000
mean,17.860300,13.983000
std,35.371233,27.282686
min,1.000000,1.000000
25%,4.000000,3.000000
50%,7.000000,6.000000
75%,14.000000,11.000000
max,735.000000,567.000000


In [ ]:
print(stats)

            rus_toks       krc_toks      rus_words      krc_words
count  260602.000000  260602.000000  260602.000000  260602.000000
mean       13.775873      18.545679       8.301982       7.806909
std        29.168188      38.949462      19.266522      17.671651
min         1.000000       1.000000       1.000000       1.000000
25%         3.000000       4.000000       1.000000       1.000000
50%         5.000000       7.000000       2.000000       2.000000
75%        11.000000      14.000000       7.000000       6.000000
max      2019.000000    2506.000000    1277.000000    1072.000000


In [ ]:
new_tot_len, tot_len = 0, 0
for text in sample:
    for tok in tokenizer.tokenize(text):
        s = len(tok)
        tot_len += s
        if tokenizer.convert_tokens_to_ids(tok) > len(tokenizer_old.sp_model):
            new_tot_len += s

print(new_tot_len / tot_len)

0.40127202549940005


# 7. UPDATING THE MODEL EMBEDDINGS ------------------------------------------------

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
print(model.model.shared)

(…)-distilled-600M/resolve/main/config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

(…)600M/resolve/main/generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Embedding(256206, 1024, padding_idx=1)


In [ ]:
model.resize_token_embeddings(len(tokenizer))
moved_tokens = list(tokenizer_old.lang_code_to_id) + ['<mask>']
model.model.shared.weight.data[tokenizer.convert_tokens_to_ids(moved_tokens)] = model.model.shared.weight.data[tokenizer_old.convert_tokens_to_ids(moved_tokens)]

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 262926. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Because we have added one more language, its id must be computed separately, e.g. as an average of related languages.  
* 'crh_Latn': 256039 - Crimean Tatar language
* 'tat_Cyrl': 256171 - Tatar language
* 'kaz_Cyrl': 256089 - Kazakh language

In [ ]:
model.model.shared.weight.data[tokenizer.convert_tokens_to_ids(LANG_UNICODE)] = (
    model.model.shared.weight.data[tokenizer_old.convert_tokens_to_ids('tat_Cyrl')] * 0.55
    + model.model.shared.weight.data[tokenizer_old.convert_tokens_to_ids('crh_Latn')] * 0.35
    + model.model.shared.weight.data[tokenizer_old.convert_tokens_to_ids('kaz_Cyrl')] * 0.1
)

re-initializing the new embeddings

In [ ]:
for t in tqdm(added_vocab):
    tt = tokenizer_old(t, add_special_tokens=False).input_ids
    if len(tt) == 0:
        tt = [tokenizer_old.unk_token_id]
    idx = tokenizer.convert_tokens_to_ids(t)
    model.model.shared.weight.data[idx] = model.model.shared.weight.data[tt].mean(0)

  0%|          | 0/6721 [00:00<?, ?it/s]

### Save Model

In [ ]:
model.save_pretrained(MODEL_PATH_RAW)
tokenizer.save_pretrained(MODEL_PATH_RAW)

('/Til/MyDrive/Colab Notebooks/Til/models/NLLB_v1/nllb_krc_raw/tokenizer_config.json',
 '/Til/MyDrive/Colab Notebooks/Til/models/NLLB_v1/nllb_krc_raw/special_tokens_map.json',
 '/Til/MyDrive/Colab Notebooks/Til/models/NLLB_v1/nllb_krc_raw/sentencepiece.bpe.model',
 '/Til/MyDrive/Colab Notebooks/Til/models/NLLB_v1/nllb_krc_raw/added_tokens.json')

In [ ]:
model1 = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH_RAW)
tokenizer1 = NllbTokenizer.from_pretrained(MODEL_PATH_RAW)
# tokenizer1 = NllbTokenizer.from_pretrained(MODEL_PATH_RAW, rebuild=True)
fix_tokenizer(tokenizer1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.model.shared
model1.model.shared

Embedding(262926, 1024, padding_idx=1)

# 8.CHECK MODEL TRANSLATION ABILITY ------------------------------------------------

translate Hindi to French

In [ ]:
article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"

tokenizer.src_lang = "hin_Deva"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["fra_Latn"],
    max_length=30,
    num_beams=1
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

["Le chef de l'ONU affirme qu'il n'y a pas de solution militaire en Syrie"]

translate Hindi to Russian

In [ ]:
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["rus_Cyrl"],
    max_length=30,
    num_beams=5,
    repetition_penalty=30.0,
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Глава ООН говорит, что в Сирии нет военного решения.']

translate Qarachay-Malqar to Russian

In [ ]:
df = all_sentences.sample(1)[['krc', 'rus']]
qm = df.krc.to_list()
ru = df.rus.to_list()

tokenizer.src_lang = LANG_UNICODE
encoded_hi = tokenizer(qm, return_tensors="pt")

generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["rus_Cyrl"],
    max_length=30,
    num_beams=5,
    repetition_penalty=30.0
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Токнуться на точку зрения.']